In [ ]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>")) # Increase cell width
display(HTML("<style>.rendered_html { font-size: 16px; }</style>")) # Increase font size

import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np

# Introduction

The exercise has been taken from the book "Introduction to Statistical Learning" (https://www.amazon.es/Introduction-Statistical-Learning-Applications-Statistics/dp/3031387465/ref=tmm_hrd_swatch_0?_encoding=UTF8&sr=8-1)



# Practice #2

Regression trees with Boston dataset

## Data Loading

For this practice we will make use of the Boston Dataset, which was included into the sklearn datasets (it is discontinued as it was catalogued as non-ethical).
<BR>
It consists of 506 rows and 14 columns. The goal is to predict the **MEDV** variable.

- CRIM - per capita crime rate by town
- ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS - proportion of non-retail business acres per town.
- CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- NOX - nitric oxides concentration (parts per 10 million)
- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940
- DIS - weighted distances to five Boston employment centres
- RAD - index of accessibility to radial highways
- TAX - full-value property-tax rate per \$10,000
- PTRATIO - pupil-teacher ratio by town
- AFRO - 1000(Bk - 0.63)^2 where Bk is the proportion of Afroamerican by town
- LSTAT - % lower status of the population
- **MEDV - Median value of owner-occupied homes in $1000's**


Let's load the dataset and split it into training and test

In [ ]:
import openpyxl

# Red data
filename = 'boston.xlsx'
df = pd.read_excel(filename, index_col=0) #it has two sheets, we load the 2nd one
df.head()

In [ ]:
df.describe()

It is not the scope of this practice to focus on other things rather than the Decision Trees themselves. So I leave to you as future work the inspecting of the dataset profiling and the feature engineering process.

Since there are not null values I will just divide the dataframe into training and test.

In [ ]:
from sklearn.model_selection import train_test_split

def split_df(dataframe, seed=None, percentage=0.8):
    
    X = dataframe.loc[:, dataframe.columns != 'MEDV']
    y = dataframe['MEDV']

    return train_test_split(X, y, test_size=1-percentage, random_state=seed)

X_train, X_test, y_train, y_test = split_df(df, seed=42, percentage=0.5)

## Single Tree

Now we can create our first decision tree. The basic DT is implemented in sklearn in the DecisionTreeRegressor method. 
<BR>You can take a look to its configuration and hyperparameters in the following link: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

For the practice we will use the default parameters.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

boston_tree = DecisionTreeRegressor(random_state=42) 
boston_tree.fit(X_train, y_train);

By default, the tree uses the squared_error (MSE), although it penalizes larger errors more, since it squares them.
<BR>The tree searches for the split that minimizes the MSE in the child nodes (minimizing the variance).
<BR>
<BR>(If we have outliers, one of the alternatives is to use the absolute_error (MAE), which is more robust to outliers because it penalizes errors linearly).


Let's take a look to the performance of the classifier.

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = boston_tree.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE = {0:.4f}".format(mse))

mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
print(f"MAPE = {mape:.2f}%")

We can also make sense of the quality of the predictions by plotting them against the actual values

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(y_test,boston_tree.predict(X_test),color='blue',label='Prediction DT')
min_val, max_val = min(y_test.min(), boston_tree.predict(X_test).min()), max(y_test.max(), boston_tree.predict(X_test).max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label="Perfect Fit (y = x)")
plt.xlabel("Actual Values", fontsize=16)
plt.ylabel("Predictions", fontsize=16)
plt.legend(loc='upper left');
plt.show()

Now we will take a look to the tree itself. We will need to make use of an external library: `pydotplus`.

In [ ]:
# ! pip install pydotplus

# ! pip install graphviz 
# you may need to install this library directly from https://graphviz.gitlab.io/_pages/Download/Download_windows.html 
# and then uncomment following two lines
#import os
#os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38/bin/'(installation folder)

from io import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

def plot_tree(tree, feature_names):
    dot_data = StringIO()
    export_graphviz(tree, out_file=dot_data, feature_names=feature_names,
                    filled=True, rounded=True,special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png("boston_train.png")
    return Image(graph.create_png())

In [ ]:
plot_tree(boston_tree, X_train.columns)

In [ ]:
# How many nodes and leaves?
print("Number of nodes: ", boston_tree.tree_.node_count)
print("Number of leaves: ", boston_tree.get_n_leaves())

Two aspects can be highlighted after taking a look at the tree:
 - The tree is huge! As we have not set any complexity pruning or max_depth we have allowed the tree to grow without any limit
 - `RM` and `LSTAT` seem to be the most important features in order to predict the value of the houses. The variable `RM` measures the number of rooms (i.e., the size of the house). The tree indicates that larger houses correspond to more expensive houses. The `LSTAT` (the percentage of individuals with lower socioeconomic status) indicates that houses in expensive neighborhoods are more expensive

We can confirm this later point by plotting the feature importance

In [ ]:
plt.figure(figsize=(8,6))
importances = boston_tree.feature_importances_
indices = np.argsort(importances)[::-1]
plt.bar(X_train.columns[indices], importances[indices]) #average reduction in MSE resulting from splitting at each node of the tree using that feature
plt.title('Feature Importance', fontsize=16);

Now we will prune the tree to see if we can improve performance.

There are different Pruning Parameters:

 - max_leaf_nodes: Reduce the number of leaf nodes
 - min_samples_leaf: Restrict the size of sample leaf. Minimum sample size in terminal nodes can be fixed to a specific value or a % of total 
 - max_depth: Reduce the depth of the tree to build a generalized tree. 

Let's focus on the depth of the tree. We will test different depth thresholds via CV (k-fold) by using the `GridSearchCV` provided by sklearn.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': range(1,16)} # 15 different depth levels

boston_tree_pruned_cv = GridSearchCV(boston_tree, 
                   param_grid,
                   scoring='neg_mean_squared_error', #negative since scikit always look to maximize metrics
                   cv=5 , n_jobs=1, verbose=1)

boston_tree_pruned_cv.fit(X_train,y_train);

In [ ]:
print("Best parameters set found on train set:")
print()
print(boston_tree_pruned_cv.best_params_)
print()
print("Grid scores:")

means = boston_tree_pruned_cv.cv_results_['mean_test_score']
stds = boston_tree_pruned_cv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, boston_tree_pruned_cv.cv_results_['params']):
    print("MSE = %0.3f (+/%0.03f) for %r" % (-mean, std * 2, params))

In [ ]:
plt.figure(figsize=(13,10))
plt.errorbar(range(1,16,1), [-m for m in means], yerr=stds, fmt='-o')
plt.title('MSE for different Depths', fontsize=20)
plt.xlabel("Depth", fontsize=16)
plt.ylabel("MSE", fontsize=16);

As can be seen in the plot, the optimal value for the depth of the decision tree is 10. However, we can find a local optimal at depth 6 (the difference does not appear to be high and the complexity is much reduced). <BR>Let's prune the tree with this value (6).

In [ ]:
boston_tree_pruned = DecisionTreeRegressor(random_state=42, max_depth=6)
tree=boston_tree_pruned.fit(X_train, y_train)
predictions = boston_tree_pruned.predict(X_test)
mse_pruned = mean_squared_error(y_test, predictions)
print(f"MSE = {mse_pruned:.2f} vs. {mse:.2f} on the non-pruned tree")

mape_pruned = np.mean(np.abs((y_test - predictions) / y_test)) * 100
print(f"MAPE = {mape_pruned:.2f}% vs. {mape:.2f}% on the non-pruned tree")

Great! We have reduced the error with a smaller tree. Let's plot it

In [ ]:
# How many nodes and leaves?
print("Number of nodes: ", boston_tree_pruned.tree_.node_count)
print("Number of leaves: ", boston_tree_pruned.get_n_leaves())

In [ ]:
from sklearn import tree

text_representation = tree.export_text(boston_tree_pruned, feature_names=list(X_test.columns))
print(text_representation)

with open("decistion_tree.log", "w") as fout:
    fout.write(text_representation)

### Tree visualizations

Visualize our pruned tree

In [ ]:
import graphviz
# DOT data
dot_data = tree.export_graphviz(boston_tree_pruned, out_file=None, 
                                feature_names=X_train.columns,  
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

In [ ]:
# Save it into a high resolution image
graph.render("boston_pruned")